# GB Network Visualization

In [49]:
import pandas as pd
import folium

Buses_combined = pd.read_csv('./buses_mapping/buses_combined.csv',dtype = {'bus385_id': str})


df_buses_501 = pd.read_csv('./buses_mapping/buses_501.csv')
df_buses_385 = pd.read_csv('./buses_mapping/buses_385.csv')
Buses_combined


,bus501_id,x_501,y_501,bus385_id,x_385,y_385,assigned_bus385_ids,assigned_x_385,assigned_y_385
0,way/975637991,-1.271945,60.293401,NaN,NaN,NaN,NaN,NaN,NaN
1,relation/8087295-275,-3.753620,58.575786,6441,-3.812256,58.546761,['6441'],[-3.812256],[58.546761]
2,way/495624575-275,-3.512715,58.564233,6442,-3.453827,58.566820,['6442'],[-3.453827],[58.56682]
3,way/567187951-275,-3.879158,58.512154,NaN,NaN,NaN,NaN,NaN,NaN
4,way/1262971878,-3.452185,58.479968,6443,-3.453827,58.490823,['6443'],[-3.453827],[58.490823]
...,...,...,...,...,...,...,...,...,...
496,way/49922626-400,-2.488599,50.623881,5274,-2.528229,50.628558,['5274'],[-2.528229],[50.628558]
497,way/26938722-400,-3.730387,50.471640,5210,-3.691406,50.477609,"['8016', '5210']","[-3.735352, -3.691406]","[50.498579, 50.477609]"
498,way/34656007-400,-4.242535,50.444878,5208,-4.244843,50.460127,['5208'],[-4.244843],[50.460127]
499,relation/9572254-400,-4.899645,50.395013,5207,-4.925995,50.368241,['5207'],[-4.925995],[50.368241]


## Add 501 buses

In [50]:
map_GB = folium.Map(location=[54, -2], zoom_start=6, tiles="CartoDB positron")

for _, row in df_buses_501.iterrows():
    
    popup_html = f"""
    <b>Bus ID:</b> {row.get("bus_id", "")}<br>
    <b>x:</b> {row.get("x", "")}<br>
    <b>y:</b> {row.get("y", "")}
    """

    folium.CircleMarker(
        location=[row["y"], row["x"]],
        radius=2,
        color="red",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=200)
    ).add_to(map_GB)

## Add units

In [51]:
df_units_capacity = pd.read_csv('../GBPower_raw/bmus_prepared.csv')
df_units = pd.read_csv('../GBPower_raw/prepared_bmus.csv')

df_units['capacity'] = df_units_capacity['capacity']
df_units

,NationalGridBmUnit,carrier,bus,lon,lat,capacity
0,IBG-EGEN1,interconnector,4969,0.6530000000000001,51.46000000000001,0.0
1,IBG-ENGI1,interconnector,4969,0.6530000000000001,51.46000000000001,0.0
2,IBG-RSQ1,interconnector,4969,0.6530000000000001,51.46000000000001,0.0
3,IVD-VKL1,interconnector,7909,-0.221,52.931,0.0
4,IVG-VKL1,interconnector,7909,-0.221,52.931,0.0
...,...,...,...,...,...,...
1060,SFERD-1,demand_flex,distributed,distributed,distributed,240.0
1061,EAS-SEL01,demand_flex,distributed,distributed,distributed,0.0
1062,SEE-PGU01,demand_flex,distributed,distributed,distributed,0.0
1063,YEP-ATL01,demand_flex,distributed,distributed,distributed,0.0


In [52]:
for _, row in df_units.iterrows():
    if row.get("lon") == 'distributed':
        continue
    popup_html = f"""
    <b>Unit:</b> {row.get("NationalGridBmUnit", "")}<br>
    <b>x:</b> {row.get("lon", "")}<br>
    <b>y:</b> {row.get("lat", "")}<br>
    <b>Capacity:</b> {row.get("capacity", "")}
    """

    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=2,
        color="purple",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=200)
    ).add_to(map_GB)

## Add Lines between buses

In [53]:
df_lines = pd.read_csv('./buses_mapping/lines_501.csv')

x = dict(zip(df_buses_501['bus_id'], df_buses_501['x']))
y = dict(zip(df_buses_501['bus_id'], df_buses_501['y']))

In [54]:
for _, row in df_lines.iterrows():
    bus0, bus1 = row['bus0'], row['bus1']
    
    try:
        coord0 = (y[bus0], x[bus0])
        coord1 = (y[bus1], x[bus1])

        folium.PolyLine(
            locations=[coord0, coord1],
            color='blue',
            weight=1.2,
            opacity=0.8,
            popup=f"{bus0} → {bus1}"
        ).add_to(map_GB)
    
    except KeyError as e:
        print(f"Missing bus in coordinate mapping: {e}")
        continue

Missing bus in coordinate mapping: 'way/264275258-275'
Missing bus in coordinate mapping: 'way/264275258-275'


## Add Lines between buses and generators

In [ ]:
import ast

x_501 = dict(zip(df_buses_501['bus_id'], df_buses_501['x']))
y_501 = dict(zip(df_buses_501['bus_id'], df_buses_501['y']))

bus_mapping = []

for _, row in Buses_combined.iterrows():
    bus501_id = row["bus501_id"]
    assigned_bus385_ids = row["assigned_bus385_ids"]
    
    if isinstance(assigned_bus385_ids, str):
        try:
            assigned_bus385_ids = ast.literal_eval(assigned_bus385_ids)
        except:
            assigned_bus385_ids = []

    if isinstance(assigned_bus385_ids, list):
        for b385_id in assigned_bus385_ids:
            bus_mapping.append({"bus385_id": b385_id, "bus501_id": bus501_id})

df_385_to_501 = pd.DataFrame(bus_mapping)
bus385_to_501 = dict(zip(df_385_to_501['bus385_id'], df_385_to_501['bus501_id']))

for _, row in df_units.iterrows():
    gen = row['NationalGridBmUnit']
    bus385_id = row['bus']
    
    try:
        bus501_id = bus385_to_501[bus385_id]

        coord0 = (y_501[bus501_id], x_501[bus501_id])  
        coord1 = (row['lat'], row['lon'])              

        folium.PolyLine(
            locations=[coord0, coord1],
            color='black',
            weight=0.8,
            opacity=0.4,
            popup=f"{gen} → {bus385_id} → {bus501_id}"
        ).add_to(map_GB)

    except KeyError as e:
        print(f"missing: {e}")
        continue


missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'
missing: 'distributed'


In [62]:
map_GB.save("GB_Network_Visualization.html")